In [1]:
from astropy.io import ascii
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.coordinates.matching import match_coordinates_3d
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import search_around_sky
from astropy.table import Table
import astropy.constants as const
from astropy.cosmology import FlatLambdaCDM
from scipy import stats
import numpy as np
import pandas as pd
from numpy.polynomial.polynomial import polyfit



cosmo = FlatLambdaCDM(H0=72,Om0=0.27)

In [2]:
mightee=Table.read("XMMLSS_polcatalogue_filtered.fits")
bg_ID = mightee['ID']
bg_redshift=np.array(mightee['best_z'])
bg_coords=SkyCoord(mightee['ra']* u.deg,mightee['dec']* u.deg)
RM_mightee=np.array(mightee['RM'])
RM_err_mightee=np.array(mightee['RM_err'])

print(len(mightee),'Background sources')
print('z =',np.min(mightee['best_z']),'-',np.max(mightee['best_z']))
print(mightee.columns)

print(np.median(mightee['RM_err']))
print(np.median(mightee['RM']))



243 Background sources
z = -99.0 - 4.1163
<TableColumns names=('ID','ra','ra_err','dec','dec_err','S_int','S_err','alpha','alpha_err','pol','pol_b','P/I','Q/I','U/I','RM','RM_err','eff_freq','RM_map','-ve_err_map','+ve_err_map','spec_z','phot_z','phot_z_uncertainty','best_z','best_z_uncertainty','pointing','distance')>
2.6
7.9


In [3]:
#MG cut
data = Table.read("XMM_GPz_MassSFR.fits")

print(data.columns)
len(data)
data = data[np.array(data['MAG_ABS_G']) <0]
len(data)
intervenors = data

<TableColumns names=('ID','RA','DEC','flux_HSC_G','flux_HSC_R','flux_HSC_I','flux_HSC_Z','flux_HSC_Y','flux_VISTA_Y','flux_VISTA_J','flux_VISTA_H','flux_VISTA_Ks','HSC_G','HSC_R','HSC_I','HSC_Z','HSC_Y','VISTA_Y','VISTA_J','VISTA_H','VISTA_Ks','z','MAG_ABS_G','MAG_ABS_R','MAG_ABS_I','MAG_ABS_Z','MAG_ABS_y','MAG_ABS_Y','MAG_ABS_J','MAG_ABS_H','MAG_ABS_Ks','AGE_BEST','MASS_BEST','SFR_BEST','SSFR_BEST')>


In [4]:
#intervenors = Table.read("XMM_GPz_MassSFR.fits")
int_ID = intervenors['ID']
int_redshift=np.array(intervenors['z'])
int_coords=SkyCoord(intervenors['RA']* u.deg,intervenors['DEC']* u.deg)
#Mg=np.array(data['MAG_ABS_G'])

print(len(intervenors),'galaxies')
print('z =',np.min(intervenors['z']),'-',np.max(intervenors['z']))

384526 galaxies
z = 0.0001 - 6.4484


In [5]:
#Calculate angular diameter for given radius(kpc)
r=266

d_a = cosmo.angular_diameter_distance(int_redshift)

angle = np.arctan(r*u.kpc / d_a).to(u.arcsec)

intervenors['angle'] = angle



In [6]:
# Set the search radius in arcseconds
search_radius = angle

# Create an empty list to store the foreground galaxies that are within the search radius of each background galaxy
matched_foreground_galaxies = []

# Loop over each background galaxy
for i, background_coord in enumerate(bg_coords):
    
    # Find the indices of the foreground galaxies that are within the search radius of the background galaxy
    sep_constraint = int_coords.separation(background_coord) < search_radius
    matched_indices = [j for j, match in enumerate(sep_constraint) if match]
    
    # If there are any foreground galaxies within the search radius, add them to the list
    if len(matched_indices) > 0:
        matched_foreground_galaxies.append(intervenors[matched_indices])

    # Print the progress every 1000 background galaxies
    if i % 10 == 0:
        print(f'Processed {i} background galaxies')

# If there are no matches, print a message
if len(matched_foreground_galaxies) == 0:
    print("No foreground galaxies were found within the search radius of any background galaxy.")

# Otherwise, convert the list of matched foreground galaxies to an astropy table and write it to a fits file
#else:
    matched_foreground_table = fits.BinTableHDU.from_columns(matched_foreground_galaxies)
#    matched_foreground_table.writeto('matched_foreground_catalog.fits', overwrite=True)

Processed 0 background galaxies
Processed 10 background galaxies
Processed 20 background galaxies
Processed 30 background galaxies
Processed 40 background galaxies
Processed 50 background galaxies
Processed 60 background galaxies
Processed 70 background galaxies
Processed 80 background galaxies
Processed 90 background galaxies
Processed 100 background galaxies
Processed 110 background galaxies
Processed 120 background galaxies
Processed 130 background galaxies
Processed 140 background galaxies
Processed 150 background galaxies
Processed 160 background galaxies
Processed 170 background galaxies
Processed 180 background galaxies
Processed 190 background galaxies
Processed 200 background galaxies
Processed 210 background galaxies
Processed 220 background galaxies
Processed 230 background galaxies
Processed 240 background galaxies


In [7]:
# Define the search radius in arcseconds
search_radius = angle

# Match the foreground galaxies to the background galaxies within the search radius
matched_indices, separation, _ = int_coords.match_to_catalog_sky(bg_coords)

# Filter out foreground galaxies that do not have a background galaxy within the search radius
good_matches = separation <= search_radius
matched_indices = matched_indices[good_matches]
foreground_matches = intervenors[good_matches]

# Create a new table for each background galaxy with its corresponding foreground galaxies
for i in range(len(mightee)):
    # Find the indices of the foreground galaxies that match the current background galaxy
    bg_matches = (matched_indices == i)
    if bg_matches.sum() == 0:
        continue  # Skip if there are no matches for this background galaxy
    
    # Get the foreground galaxies that match the current background galaxy
    fg_matches = foreground_matches[bg_matches]
    
    # Write the foreground galaxy table to a new file with the name "foreground_catalog_bg_{i}.fits"
#    filename = f'foreground_catalog_bg_{i}.fits'
#    fg_matches.write(filename, overwrite=True)
    
    print(f"Processed {len(fg_matches)} foreground galaxies for background galaxy {i}")
    

Processed 99 foreground galaxies for background galaxy 0
Processed 70 foreground galaxies for background galaxy 1
Processed 75 foreground galaxies for background galaxy 2
Processed 39 foreground galaxies for background galaxy 3
Processed 49 foreground galaxies for background galaxy 4
Processed 79 foreground galaxies for background galaxy 5
Processed 40 foreground galaxies for background galaxy 6
Processed 40 foreground galaxies for background galaxy 7
Processed 57 foreground galaxies for background galaxy 8
Processed 91 foreground galaxies for background galaxy 9
Processed 55 foreground galaxies for background galaxy 10
Processed 8 foreground galaxies for background galaxy 11
Processed 30 foreground galaxies for background galaxy 12
Processed 41 foreground galaxies for background galaxy 13
Processed 47 foreground galaxies for background galaxy 14
Processed 22 foreground galaxies for background galaxy 15
Processed 59 foreground galaxies for background galaxy 16
Processed 29 foreground g

Processed 37 foreground galaxies for background galaxy 174
Processed 50 foreground galaxies for background galaxy 175
Processed 38 foreground galaxies for background galaxy 176
Processed 61 foreground galaxies for background galaxy 177
Processed 92 foreground galaxies for background galaxy 178
Processed 34 foreground galaxies for background galaxy 179
Processed 96 foreground galaxies for background galaxy 180
Processed 40 foreground galaxies for background galaxy 181
Processed 51 foreground galaxies for background galaxy 182
Processed 49 foreground galaxies for background galaxy 183
Processed 44 foreground galaxies for background galaxy 184
Processed 18 foreground galaxies for background galaxy 185
Processed 60 foreground galaxies for background galaxy 186
Processed 16 foreground galaxies for background galaxy 187
Processed 17 foreground galaxies for background galaxy 188
Processed 57 foreground galaxies for background galaxy 189
Processed 38 foreground galaxies for background galaxy 1

In [ ]:
# Define the search radius in arcseconds
search_radius = angle


# Match the foreground galaxies to the background galaxies within the search radius
matched_indices, separation, _ = int_coords.match_to_catalog_sky(bg_coords)

# Filter out foreground galaxies that do not have a background galaxy within the search radius
good_matches = separation <= search_radius
matched_indices = matched_indices[good_matches]

# Count the number of intervening galaxies to each background galaxy
num_intervening = []
for i in range(len(mightee)):
    # Find the indices of the foreground galaxies that match the current background galaxy
    bg_matches = (matched_indices == i)
    if bg_matches.sum() == 0:
        num_intervening.append(0)  # No matches for this background galaxy
        continue
    
    # Count the number of foreground galaxies that match the current background galaxy
    num_matches = bg_matches.sum()
    num_intervening.append(num_matches - 1)  # Exclude the background galaxy itself
    
    # Get the foreground galaxies that match the current background galaxy
    fg_matches = intervenors[good_matches][bg_matches]
    
    # Check if intervening galaxies lie in the foreground to the background galaxy
    bg_redshift = mightee['best_z'][i]
    intervening_matches = fg_matches[fg_matches['z'] < bg_redshift]
    
    # Write the foreground galaxy table to a new file with the name "foreground_catalog_bg_{i}.fits"
#    filename = f'foreground_catalog_bg_{i}.fits'
#    intervening_matches.write(filename, overwrite=True)
    
    print(f"Processed {len(intervening_matches)} intervening galaxies for background galaxy {i}")

# Add the number of intervening galaxies as a new column to the background catalog
mightee['num_intervening'] = num_intervening

# Write the updated background catalog to a new file
#background_catalog.write('background_catalog_with_intervening.fits', overwrite=True)


Processed 0 intervening galaxies for background galaxy 0
Processed 66 intervening galaxies for background galaxy 1
Processed 61 intervening galaxies for background galaxy 2
Processed 16 intervening galaxies for background galaxy 3
Processed 9 intervening galaxies for background galaxy 4
Processed 72 intervening galaxies for background galaxy 5
Processed 29 intervening galaxies for background galaxy 6
Processed 18 intervening galaxies for background galaxy 7
Processed 34 intervening galaxies for background galaxy 8
Processed 0 intervening galaxies for background galaxy 9
Processed 0 intervening galaxies for background galaxy 10
Processed 5 intervening galaxies for background galaxy 11
Processed 0 intervening galaxies for background galaxy 12
Processed 0 intervening galaxies for background galaxy 13
Processed 29 intervening galaxies for background galaxy 14
Processed 7 intervening galaxies for background galaxy 15
Processed 56 intervening galaxies for background galaxy 16
Processed 8 int

Processed 0 intervening galaxies for background galaxy 141
Processed 0 intervening galaxies for background galaxy 142
Processed 0 intervening galaxies for background galaxy 143
Processed 0 intervening galaxies for background galaxy 144
Processed 45 intervening galaxies for background galaxy 145
Processed 21 intervening galaxies for background galaxy 146
Processed 34 intervening galaxies for background galaxy 147
Processed 48 intervening galaxies for background galaxy 148
Processed 24 intervening galaxies for background galaxy 149
Processed 35 intervening galaxies for background galaxy 150
Processed 0 intervening galaxies for background galaxy 151
Processed 0 intervening galaxies for background galaxy 152
Processed 0 intervening galaxies for background galaxy 153
Processed 29 intervening galaxies for background galaxy 154
Processed 72 intervening galaxies for background galaxy 155
Processed 0 intervening galaxies for background galaxy 156
Processed 24 intervening galaxies for background

In [ ]:
#MEDIAN binning
plt.figure(figsize=(10, 10), dpi=200)
#bin_std, bin_edges, binnumber = stats.binned_statistic(separations[separations<400],np.absolute(RM[separations<400]),statistic='std',bins=3)
#bin_median, bin_edges, binnumber = stats.binned_statistic(separations[separations<400],np.absolute(RM[separations<400]),statistic='median',bins=3)
#bin_width = (bin_edges[1] - bin_edges[0])
#bin_centers = bin_edges[1:] - bin_width/2


plt.scatter(mightee['num_intervening'],np.absolute(mightee['RM']) ,marker='o', color='red')

#plt.errorbar(mightee['num_intervening'],np.absolute(mightee['RM']), mightee['RM_err']*0.5, fmt='none', color='black', linewidth=0.5)

b, m = polyfit(mightee['num_intervening'][np.absolute(mightee['RM'])<100], np.absolute(mightee['RM'][np.absolute(mightee['RM'])<100]), 1)
plt.plot(mightee['num_intervening'][np.absolute(mightee['RM'])<100], b + m * mightee['num_intervening'][np.absolute(mightee['RM'])<100], linestyle='solid', color='black')


#plt.plot(np.unique(separations[separations<400]), np.poly1d(np.polyfit(separations[separations<400], np.absolute(RM)[separations<400], 1))(np.unique(separations[separations<400])),color='black', linestyle="dashed")
#plt.hlines(bin_median, bin_edges[:-1], bin_edges[1:], color='green', lw=10)
#numberofpoints=[sum(binnumber==i) for i in np.arange(1,4)]
#plt.hlines(3.425,-5,405, linestyle='dashed')
#plt.vlines(bin_centers,bin_median-bin_std/np.sqrt(numberofpoints),bin_median+bin_std/np.sqrt(numberofpoints), colors='blue', lw=5)
plt.xlabel('N$_{int}$', fontsize=20)
plt.ylabel('|RM| [rad m$^{-2}$]', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.xlim(-5,400)
plt.ylim(-1,23.5)
plt.text(70,22.5, 'XMM-LSS',fontsize=20)


plt.savefig('XMM_N_int_RM_133kpc.png', bbox_inches='tight')
plt.show()

#print(bin_median)
#print(bin_centers)
#print(np.std(bin_median))
#print(bin_median-bin_std/np.sqrt(numberofpoints))
#print(bin_median+bin_std/np.sqrt(numberofpoints))

In [ ]:
from scipy.stats import pearsonr

# Input data points
x = mightee['num_intervening'][np.absolute(mightee['RM'])<23]
y = np.absolute(mightee['RM'][np.absolute(mightee['RM'])<23])

# Linear fit
coefficients = np.polyfit(x, y, 1)
a, b = coefficients

# Linear fit function
def linear_fit(x):
    return a*x + b

# Pearson test
r, p = pearsonr(y, [linear_fit(i) for i in x])
print("Pearson correlation coefficient:", r)
print("p-value:", p)

In [ ]:
# Read in the background and foreground catalogs
background_catalog = Table.read('COSMOS_1587911796_1524147354_1525613583_pol_detections.fits')
foreground_catalog = Table.read('COSMOS_GPz_MassSFR.fits')

# Convert the RA and DEC columns to SkyCoord objects
background_coords = SkyCoord(ra=background_catalog['ra']*u.degree,
                             dec=background_catalog['dec']*u.degree)

foreground_coords = SkyCoord(ra=foreground_catalog['RA']*u.degree,
                             dec=foreground_catalog['DEC']*u.degree)


# Create an empty array to store the median separations
median_seps = []

# Loop over all the background sources
for i in range(len(background_catalog)):
    
    # Get the coordinates of the current background source
    current_coord = background_coords[i]
    
    # Find all the foreground sources within certain angle of the current background source
    sep_constraint = current_coord.separation(foreground_coords) < angle
    intervening_sources = foreground_catalog[sep_constraint]
    
    # Check if intervening galaxies lie in the foreground to the background galaxy
    bg_redshift = mightee['best_z'][i]
    intervening_matches = fg_matches[fg_matches['z'] < bg_redshift]
    
    # Calculate the separations between the current background source and all intervening sources
    separations = current_coord.separation(SkyCoord(ra=intervening_sources['RA']*u.degree,
                                                    dec=intervening_sources['DEC']*u.degree))
    
    print(separations)
    
    # Take the median of the separations
#    median_sep = separations.median()
    
    # Append the median separation to the list
#    median_seps.append(median_sep)

# Add the median separation column to the background catalog
#background_catalog['Median Separation'] = median_seps

# Save the updated background catalog
#background_catalog.write('background_catalog_with_median_sep.fits', overwrite=True)

In [ ]:
comoving_distance = cosmo.comoving_distance(fg_matches['z'])
for i in separations:
    angle_rad = i.to(u.rad).value
    physical_distance = comoving_distance * angle_rad
    
physical_distance

In [ ]:
# Define the search radius in arcseconds
search_radius = angle


# Match the foreground galaxies to the background galaxies within the search radius
matched_indices, separation, _ = int_coords.match_to_catalog_sky(bg_coords)

# Filter out foreground galaxies that do not have a background galaxy within the search radius
good_matches = separation <= search_radius
matched_indices = matched_indices[good_matches]

# Create empty list for separations
median_seps = []

# Count the number of intervening galaxies to each background galaxy
num_intervening = []
for i in range(len(mightee)):
    # Find the indices of the foreground galaxies that match the current background galaxy
    bg_matches = (matched_indices == i)
    if bg_matches.sum() == 0:
        num_intervening.append(0)  # No matches for this background galaxy
        continue
    
    # Count the number of foreground galaxies that match the current background galaxy
    num_matches = bg_matches.sum()
    num_intervening.append(num_matches - 1)  # Exclude the background galaxy itself
    
    # Get the foreground galaxies that match the current background galaxy
    fg_matches = intervenors[good_matches][bg_matches]
    
    # Check if intervening galaxies lie in the foreground to the background galaxy
    bg_redshift = mightee['best_z'][i]
    intervening_matches = fg_matches[fg_matches['z'] < bg_redshift]
    
    # Calculate the separations between the current background source and all intervening sources
    separations = current_coord.separation(SkyCoord(ra=intervenors['RA']*u.degree,
                                                    dec=intervenors['DEC']*u.degree))
    
    # Append the separation to the list
    median_seps.append(separations)
    
    # Write the foreground galaxy table to a new file with the name "foreground_catalog_bg_{i}.fits"
#    filename = f'foreground_catalog_bg_{i}.fits'
#    intervening_matches.write(filename, overwrite=True)
    
    print(f"Processed {len(intervening_matches)} intervening galaxies for background galaxy {i}")

# Add the number of intervening galaxies and separation as a new column to the background catalog
mightee['num_intervening'] = num_intervening

# Write the updated background catalog to a new file
#background_catalog.write('background_catalog_with_intervening.fits', overwrite=True)

In [ ]:
mightee['median_seps']